In [113]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import datetime

In [114]:
data = pd.read_csv('dataframe/champions.csv')

In [115]:
data['Temporada'] = pd.to_datetime(data['Temporada'])

# Ordenar el DataFrame por la columna 'Temporada'
data_sorted = data.sort_values(by='Temporada')

In [116]:
equipo_de_interes = [1, 3, 5, 14, 16, 24, 28, 31]
equipo_data = data[data['Local'].isin(equipo_de_interes)]

serie_temporal = equipo_data.set_index('Temporada')['GolesLocal']

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
train_size = int(len(serie_temporal) * 0.8)
train_data, test_data = serie_temporal.iloc[:train_size], serie_temporal.iloc[train_size:]

# Convertir los datos de entrenamiento y prueba en DataFrames
train = pd.DataFrame({'Temporada': train_data.index, 'GolesLocal': train_data.values})
test = pd.DataFrame({'Temporada': test_data.index, 'GolesLocal': test_data.values})

# Establecer el índice de tiempo en los DataFrames
train = train.set_index('Temporada')
test = test.set_index('Temporada')



In [117]:
print(train.columns)


Index(['GolesLocal'], dtype='object')


In [118]:
def arima_model(train, test, order):
    # Selección de la columna 'GolesLocal' de los datos de entrenamiento
    train_data = train['GolesLocal']
    
    # Crear el modelo ARIMA
    model = ARIMA(train_data, order=order)
    
    # Ajustar el modelo a los datos
    model_fit = model.fit()
    
    # Predecir los valores futuros
    forecast = model_fit.forecast(steps=len(test))
    
    return model_fit, forecast


In [119]:
data.head()

,idPartido,Temporada,Ronda,Local,Visitante,Evento,GolesLocal,GolesVisitante,VictoriaLocal,Empate,...,1_MediaJugadores,%_2_G_Temporada,%_2_G_Temporada_L,%_2_E_Temporada_L,%_2_P_Temporada_L,2_Media_G,2_Media_G_Local,2_Media_Goles_PP,2_ValorJugadores,2_MediaJugadores
0,121,2022-01-01,Group stage,235,11,Stadion Maksimir,1,0,1,0,...,18.743616,55.56,60.00,20.00,20.00,0.56,0.60,1.33,685.188968,26.353422
1,122,2022-01-01,Group stage,14,153,Signal Iduna Park,3,0,1,0,...,19.853461,0.00,0.00,100.00,0.00,0.00,0.00,0.20,356.128703,14.245148
2,123,2022-01-01,Group stage,15,1,Celtic Park,0,3,0,0,...,18.490575,63.64,83.33,16.67,0.00,0.64,0.83,2.09,796.271706,36.194168
3,124,2022-01-01,Group stage,10,298,Estádio do Sport Lisboa e Benfica,2,0,1,0,...,28.153728,20.00,33.33,0.00,66.67,0.20,0.33,1.40,382.737487,19.136874
4,125,2022-01-01,Group stage,184,2,Red Bull Arena,1,1,0,1,...,19.353798,45.45,66.67,0.00,33.33,0.45,0.67,1.27,749.233912,32.575387


In [120]:
import matplotlib.pyplot as plt

def series_plot(data, title):
    plt.figure(figsize=(12, 8))  # Ajustar el tamaño de la figura

    for equipo in equipo_de_interes:
        # Filtrar los datos para el equipo actual
        equipo_data = data[data['Local'] == equipo]
        
        # Agrupar los datos por Temporada y sumar los goles marcados como local para cada temporada
        serie_temporal = equipo_data.groupby('Temporada')['GolesLocal'].sum()
        
        # Dibujar la serie temporal del equipo actual
        plt.plot(serie_temporal.index, serie_temporal.values, marker='o', linestyle='-', label=f'Equipo {equipo}')

    plt.title('Series Temporales de Goles Marcados como Local por Equipo')
    plt.xlabel('Temporada')
    plt.ylabel('Goles Marcados')
    plt.xticks(rotation=45)
    plt.legend()  # Mostrar leyenda con etiquetas de equipo
    plt.grid(True)
    plt.tight_layout()
    plt.show()

series_plot(serie_temporal, 'Serie Temporal de Goles Marcados')

KeyError: 'Local'

<Figure size 1200x800 with 0 Axes>

In [ ]:
def evaluacion_modelo(test, forecast):
    errores = test - forecast

    # Métricas de evaluación
    mse = np.mean(errores ** 2)
    mae = np.mean(np.abs(errores))
    rmse = np.sqrt(mse)
    
    # Diccionario con las métricas
    metricas = {
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse
    }
    
    return metricas

In [ ]:
arima_model(train, test, order=(1, 1, 1))


/Users/carlotasanchezgonzalez/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/carlotasanchezgonzalez/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/carlotasanchezgonzalez/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/carlotasanchezgonzalez/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so w

(<statsmodels.tsa.arima.model.ARIMAResultsWrapper at 0x156a13450>,
 464    2.369016
 465    2.379070
 466    2.379344
 467    2.379352
 468    2.379352
          ...   
 576    2.379352
 577    2.379352
 578    2.379352
 579    2.379352
 580    2.379352
 Name: predicted_mean, Length: 117, dtype: float64)